# Presentación GloVe
- Fernanda Rubio
- Roberto Pérez 
- Víctor Rivera

## Introducción

Existen dos familias de algoritmos para "aprender" vectores de palabras

- **Global Matrix Factorization** (e.g. LSA)

    Ventaja: Aprovechan información estadística

- **Local Context Widow** (e.g. skip-gram)

    Ventaja: Buen desempeño encontrando analogías de palabras

**GloVe** (Global Vectors) combina las ventajas de las 2 familias de modelos   

## GloVe

GloVe es un algoritmo *no supervisado* para obtener representaciones vectoriales de palabras.

Sea:
- $X$ la matriz de coocurrencia.
- $X_{i,j}$ el número de veces la palabra j ocurre en el contexto de la palabra i.
- $X_{i} = \sum_{k}X_{i,k}$ el número de veces que la palabra i ocurre en el contexto de cualquier palabra.
- $P_{i,j} = P(j|i) = \frac{X_{i,k}}{X_{j,k}}$ la probabilidad de que j aparezca en el contexto de i.

Para 2 palabras $i$, $j$ y un set de palabras $k$ de contexto, su relación se estudia por el ratio de sus coocurrencias:

  * para palabras relacionadas a i pero no a j $\frac{P_{i,k}}{P_{j,k}}$ será grande.
  * para palabras relacionadas a j pero no a i  $\frac{P_{i,k}}{P_{j,k}}$ será pequeño.
  * para palabras relacionadas por igual a i y j o no relacionadas $\frac{P_{i,k}}{P_{j,k}}$ será cercano a 1.

<img src="https://docs.google.com/uc?export=download&id=1hisLB3xsrAkDnkz2J15M3O-JiYE_q39B">

Comparar estos ratios nos permite saber qué palabras son relevantes y a discriminar para qué palabra son relevantes. Entonces el principio de GloVe es una función que depende de 3 palabras i, j y k:
$$F(w_{i}, w_{j}, \hat{w}_{k})=\frac{P_{i,k}}{P_{j,k}}$$

donde: 
- $w_{i}, w_{j}$ son vectores de palabras
- $\hat{w}_{k}$ vectores de palabras de contexto
- $F$ puede ser cualquier función pero nos gustaría que capturara más información de $\frac{P_{i,k}}{P_{j,k}}$.

Planteando el problema como uno de mínimos cuadrados, el objetivo GloVe se vuelve minimizar la siguiente función de pérdida:
$$J=\sum_{i,j=1}^{V}f(X_{ij})(w_{i}^{t}\hat{w}_{k}+b_{i}+\hat{b}_{k}-ln(X_{ik}))^2$$

## Ejemplo aplicado

Cargamos una muestra de artículos de Wikipedia:

In [ ]:
wiki_corp <- quanteda.corpora::download(
    url = "https://www.dropbox.com/s/9mubqwpgls3qi9t/data_corpus_wiki.rds?dl=1"
)

Creación de vocabulario del que se aprenderán los vectores de palabras:

1: Tokenizar el corpus:

In [ ]:
wiki_toks <- tokens(wiki_corp)

2: Extración de los features que suceden 5 veces o más:

In [ ]:
feats <- dfm(wiki_toks, verbose = TRUE) %>%
    dfm_trim(min_termfreq = 5) %>%
    featnames()

<img src="https://docs.google.com/uc?export=download&id=1Xd6o5c-VbRw3qbJeAWuKbIHEwZAiTUme">

In [ ]:
wiki_toks <- tokens_select(wiki_toks, feats, padding = TRUE)

Contrucción de la matriz de concurrencia:

In [ ]:
wiki_fcm <- fcm(wiki_toks, context = "window", count = "weighted", weights = 1 / (1:5), tri = TRUE)
wiki_fcm

### Modelo GloVe

Entrenamiento del modelo

In [ ]:
glove <- GlobalVectors$new(rank = 50, x_max = 10)
wv_main <- glove$fit_transform(wiki_fcm, n_iter = 10,
                               convergence_tol = 0.01, n_threads = 8)

<img src="https://docs.google.com/uc?export=download&id=1rWhZehCPj-fkioj39sv7qEbOsD5hPcds">

Suma de palabras con el contexto para mejorar la precisión:

In [ ]:
wv_context <- glove$components
word_vectors <- wv_main + t(wv_context)

### Entrenar word2vec

In [ ]:
normalizar <- function(texto, vocab = NULL){
  # minúsculas
  texto <- tolower(texto)
  # varios ajustes
  texto <- gsub("\\s+", " ", texto)
  texto <- gsub("\\.[^0-9]", " _punto_ ", texto)
  texto <- gsub(" _s_ $", "", texto)
  texto <- gsub("\\.", " _punto_ ", texto)
  texto <- gsub("[«»¡!¿?-]", "", texto) 
  texto <- gsub(";", " _punto_coma_ ", texto) 
  texto <- gsub("\\:", " _dos_puntos_ ", texto) 
  texto <- gsub("\\,[^0-9]", " _coma_ ", texto)
  texto <- gsub("\\s+", " ", texto)
  texto
}
wiki_df <- tibble(txt = wiki_corp) %>%
                mutate(id = row_number()) %>%
                mutate(txt = normalizar(txt))

if(!file.exists('./salidas/wiki_w2v.txt')){
  tmp <- tempfile()
  # tokenización
  write_lines(wiki_df$txt,  tmp)
  prep <- prep_word2vec(tmp, 
          destination = './salidas/wiki_w2v.txt', bundle_ngrams = 2)
} 

if (!file.exists("./salidas/wiki_vectors.bin")) {
  model_w2v <- train_word2vec("./salidas/wiki_w2v.txt", 
          "./salidas/wiki_vectors.bin",
          vectors = 100, threads = 4, window = 5, cbow = 0,  
          iter = 5, negative_samples = 20, min_count = 5) 
} else {
  model_w2v <- read.vectors("./salidas/wiki_vectors.bin")
}

### Datos de prueba

In [ ]:
test_data <- read_delim("words_for_testing.csv", delim = " ", col_names = F) %>%
  rename(group = X5, word_to_predict = X3) %>%
  select(X1, X2, X4, word_to_predict, group) %>%
  mutate_all(tolower) %>%

filter(X1 %in% rownames(word_vectors), 
       X2 %in% rownames(word_vectors), 
       X4 %in% rownames(word_v ectors)
)

<img src="https://docs.google.com/uc?export=download&id=1iAHMokUtnpo6pFxFe6EIJgahS_dSbVcK">

### Comparación

Esta función calcula los vectores de diferencia y obtiene la palabra con mayor similitud para cada modelo:

In [ ]:
similarity_comp <- function(test_data){ for (i in 1:nrow(test_data)) {
    vec_glove <- word_vectors[test_data$X1[i], , drop = FALSE] -
      word_vectors[test_data$X2[i], , drop = FALSE] +
      word_vectors[test_data$X4[i], , drop = FALSE]
    vec_w2v <- model_w2v[[test_data$X1[i]]] - model_w2v[[test_data$X2[i]]] +
      model_w2v[[test_data$X4[i]]]
    cos_sim <- textstat_simil(x = as.dfm(word_vectors), y = as.dfm(vec_glove), method = "cosine"
)
    order_words <- head(sort(cos_sim[, 1], decreasing = TRUE), 5)
    closest_word_glove <- order_words %>% as_tibble() %>% mutate(word = names(order_words)) %>%
      filter(word != test_data$X4[i])
    closest_word_w2v <- model_w2v %>% closest_to(vec_w2v, n = 5) %>%
      filter(word != test_data$X4[i])
    test_data$prediction_glove[i] <- closest_word_glove$word[1]
    test_data$conf_glove[i] <- closest_word_glove$value[1]
    test_data$prediction_w2v[i] <- closest_word_w2v$word[1]
    test_data$conf_w2v[i] <- closest_word_w2v$`similarity to vec_w2v`[1]
}
test_data }

### Prueba con datos de familia

Estos datos reflejan relaciones familiares:

In [ ]:
fam_test_data <- test_data %>%
  filter(group == 'family') %>%
  select(-group)
head(fam_test_data)

<img src="https://docs.google.com/uc?export=download&id=1HKwv5uL8R0b8fvUfh8r6tK57mkZ4Il7N">

Aplicamos la función:

In [ ]:
fam_test_data <- fam_test_data %>%
  mutate(prediction_glove = NA, conf_glove = NA, prediction_w2v = NA, conf_w2v = NA)
fam_test_data <- similarity_comp(fam_test_data)

Vemos en cuántas ocasiones el modelo dio la palabra que estabamos buscando, en este caso el performance de GloVe es mejor que el de word2vec:

In [ ]:
fam_eval <- fam_test_data %>%
  mutate(equal_glove = if_else(word_to_predict == prediction_glove, 1, 0),
         equal_w2v = if_else(word_to_predict == prediction_w2v, 1, 0))
fam_eval %>% filter(equal_glove == 1) %>% head(10)

<img src="https://docs.google.com/uc?export=download&id=13oW_DX6tVDGWAAgFeJ-1zQGS8Gb0XoVB">

In [ ]:
fam_eval %>%
  summarise(sum(equal_glove), sum(equal_w2v))

<img src="https://docs.google.com/uc?export=download&id=1nQhyUQoZ-sL4BwZs8UYROvO8uNr855T7">

### Prueba con datos de capitales del mundo

In [ ]:
country_test_data <- test_data %>%
  filter(group == 'capital-world') %>%
  select(-group)
head(country_test_data)

<img src="https://docs.google.com/uc?export=download&id=1PWt1sxs3jMKS_MZPUNELtfeBVxUJoJDw">

In [ ]:
country_test_data <- country_test_data %>%
  mutate(prediction_glove = NA, conf_glove = NA, prediction_w2v = NA, conf_w2v = NA)
country_test_data <- similarity_comp(country_test_data)

Vemos en cuántas ocasiones el modelo dio la palabra que estabamos buscando, en este caso el performance de GloVe es mejor que el de word2vec:

In [ ]:
country_eval <- country_test_data %>%
  mutate(equal_glove = if_else(word_to_predict == prediction_glove, 1, 0),
         equal_w2v = if_else(word_to_predict == prediction_w2v, 1, 0))
country_eval %>% filter(equal_glove == 1) %>% head(10)

<img src="https://docs.google.com/uc?export=download&id=1Gztn0C2HwdWgIXq2Hv3fxYRsJicTnDcB">

In [ ]:
country_eval %>%
 summarise(sum(equal_glove), sum(equal_w2v))

<img src="https://docs.google.com/uc?export=download&id=1wptwnowiykhxQRl15bbCN19S1rg-yobp">

## Gracias